In [1]:
# source: https://github.com/pytorch/examples/blob/master/mnist/main.py

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=14, metavar='N',
                    help='number of epochs to train (default: 14)')
parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                    help='learning rate (default: 1.0)')
parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                    help='Learning rate step gamma (default: 0.7)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')

parser.add_argument('--save-model', action='store_true', default=True,
                    help='For Saving the current Model')
args = parser.parse_args(args=[])
use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")
print(device)

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor()])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor()])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)
    scheduler.step()

if args.save_model:
    torch.save(model.state_dict(), "/content/gdrive/My Drive/project_321089724_302363965/MNIST Inception Score/mnist_cnn2.pt")

  0%|          | 0/9912422 [00:00<?, ?it/s]

cuda


9920512it [00:00, 1603392.97it/s]                             


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


32768it [00:00, 317705.20it/s]                           
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


1654784it [00:00, 5218221.08it/s]                           
8192it [00:00, 130333.19it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312773
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.405952
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.858182
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.779474
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.449638
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.460928
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.465206
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.321517
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.649814
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.317983
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.471585
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.272673
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.272987
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.283677
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.407331
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.182749
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.130984
Train E

In [0]:
import numpy as np
batchSize = 1000
## load images

kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                  transform=transforms.Compose([
                      transforms.ToTensor()])),
    batch_size=batchSize, shuffle=True, **kwargs)
for batch_idx, (data, target) in enumerate(train_loader):
    images, target = data.to(device), target.to(device)
    break

preds = torch.exp(model(images)).detach().cpu().numpy()
print(np.mean(np.argmax(preds,axis=1)==target.cpu().numpy()))

0.999


In [0]:
torch.save(model.state_dict(),F"/content/gdrive/My Drive/project_321089724_302363965/MNIST Inception Score/mnist_cnn.pt")

In [0]:
from glob import glob
model2 = Net()
print(model2.conv1.bias)
path = F"/content/gdrive/My Drive/project_321089724_302363965/MNIST Inception Score/mnist_cnn.pt"
filename = glob(path)[0]
print(filename)
model2.load_state_dict(torch.load(filename))
print(model.conv1.bias)
print(model2.conv1.bias)

Parameter containing:
tensor([ 0.2675, -0.1661, -0.0187,  0.1470,  0.1990,  0.0672,  0.0657, -0.0958,
         0.1738, -0.2058, -0.2390, -0.2098,  0.0577, -0.1355,  0.2519,  0.0783,
        -0.2697,  0.1060,  0.0563,  0.1234,  0.0704, -0.2696, -0.2540, -0.0455,
        -0.2355, -0.0375,  0.0130, -0.2067,  0.1766, -0.1189,  0.2166,  0.2445],
       requires_grad=True)
/content/gdrive/My Drive/project_321089724_302363965/MNIST Inception Score/mnist_cnn.pt
Parameter containing:
tensor([-0.1881, -0.0616, -0.0656, -0.2687, -0.0881, -0.0027, -0.1847, -0.2662,
        -0.1588, -0.1416, -0.2062, -0.3213, -0.2772, -0.2592, -0.2459, -0.1633,
        -0.2036, -0.2812, -0.2949, -0.0867, -0.0691,  0.0633, -0.1348, -0.3577,
        -0.3189, -0.2736, -0.1337, -0.2253, -0.2807, -0.0538, -0.0788, -0.1264],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.1881, -0.0616, -0.0656, -0.2687, -0.0881, -0.0027, -0.1847, -0.2662,
        -0.1588, -0.1416, -0.2062, -0.3213, -0.2772, 